# Non-DC ML water masks creation

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time
import os

from subprocess import Popen, PIPE, STDOUT

from utils.genprepWater import per_scene_wofs
from utils.prep_utils import s3_list_objects_paths

from utils.genprepMLWater import genprepmlwater

In [7]:
# # should encode to env vars as with docker, not csv
# os.environ['AWS_ACCESS_KEY_ID']= str(pd.read_csv('../aws_creds.csv').AWSAccessKeyId.values[0])
# os.environ['AWS_SECRET_ACCESS_KEY']= str(pd.read_csv('../aws_creds.csv').AWSSecretKey.values[0])
# # os.environ['AWS_S3_ENDPOINT']=str(pd.read_csv('../aws_creds.csv').AWS_S3_ENDPOINT.values[0])
print(os.environ['AWS_ACCESS_KEY_ID'])
print(os.environ['AWS_S3_ENDPOINT'])

16c353e3c1673eb9556e
s3-uk-1.sa-catapult.co.uk


# **Example job**

essential **input vars**

In [8]:
# essential input vars
s3_bucket = 'public-eo-data'
s3_dir = 'common_sensing/tom/mlperscene_test2/'
inter_dir = '../data/'

find **list of scene yamls** - this is essentially input to the job list

In [9]:
s3_dir_allprods = 'common_sensing/fiji/'
# prep to single yaml path
s3_paths = s3_list_objects_paths(s3_bucket, s3_dir_allprods)
yamls = [i for i in s3_paths if i.split('/')[-1][-4:] == 'yaml']
len(yamls)

11680

filter to **yamls of test scenes** - i.e. scenes from desired sensors that are largely spatially covered by and within preceeding year to Annual Wofs Water Summary test aoi and year

In [10]:
ys = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
print("YEAR  |  L7  |  L8  |  S2  |  S1")
for y in ys:
    l7_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'landsat_7') & ('074072' in i) & (y == i.split('/')[3].split('_')[-1][:4])])
    l8_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'landsat_8') & ('074072' in i) & (y == i.split('/')[3].split('_')[-1][:4])])
    s2_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'sentinel_2') & ('60KXF' in i ) & (y in i)])
    s1_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'sentinel_1') & (y in i)])
    
    print(y, len(l7_ymls), len(l8_ymls), len(s2_ymls), len(s1_ymls))

YEAR  |  L7  |  L8  |  S2  |  S1
2000 6 0 0 0
2001 11 0 0 9
2002 7 0 0 3
2003 7 0 0 0
2004 13 0 0 0
2005 12 0 0 0
2006 14 0 0 0
2007 8 0 0 0
2008 11 0 0 0
2009 10 0 0 0
2010 16 0 0 0
2011 7 0 0 0
2012 15 0 0 0
2013 14 15 0 0
2014 0 22 0 0
2015 0 23 0 0
2016 0 22 0 0
2017 0 23 0 69
2018 0 23 34 86
2019 0 16 82 86
2020 0 0 0 12


In [11]:
l8_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'landsat_8') & ('074072' in i) & ('2019' == i.split('/')[3].split('_')[-1][:4])])
s2_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'sentinel_2') & ('60KXF' in i ) & ('2019' in i)])
len(l8_ymls), len(s2_ymls)

(16, 82)

In [12]:
s1_ymls = sorted([i for i in yamls if (i.split('/')[2] == 'sentinel_1') & ('2019' in i)])
len(s1_ymls)

86

In [13]:
# l8_ymls
# s2_ymls
# s1_ymls

filter yamls to find list of **available annual water summary** products

In [14]:
for i in l8_ymls: print(i)

common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190113/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190129/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190214/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190302/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190318/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190403/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190419/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190505/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190521/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190606/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190622/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190708/datacube-metadata.yaml
common_sensing/fiji/landsat_8/LC08_L1TP_074072_20190

In [15]:
wofssummary_ymls = [i for i in yamls if (i.split('/')[2] == 'wofs_summary') & ('2013' in i)]
len(wofssummary_ymls)

1

example **job function**

In [16]:
for sc in l8_ymls:
    genprepmlwater(sc, wofssummary_ymls[0], s3_bucket=s3_bucket, s3_dir=s3_dir, inter_dir=inter_dir)

LANDSAT_8 ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 2
building tree 2 of 2


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   25.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.6s finished


Preparing scene LC08_L1TP_074072_20190113_mlwater
Scene path ../data/LC08_L1TP_074072_20190113_tmp/LC08_L1TP_074072_20190113_mlwater/
{'watermask': {'path': 'LC08_L1TP_074072_20190113_watermask.tif'}, 'waterprob': {'path': 'LC08_L1TP_074072_20190113_waterprob.tif'}}
{'spatial_reference': 'PROJCS["WGS 84 / UTM zone 60N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",177],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32660"]]', 'geo_ref_points': {'ul': {'x': 585300.0, 'y': -1802100.0}, 'ur': {'x': 816000.0, 'y': -1802100.0}, 'll': {'x'

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 2building tree 2 of 2



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   23.6s finished
ERROR:root:LC08_L1TP_074072_20190129 Prediction or re-shaping failed
Traceback (most recent call last):
  File "/home/dataprep/ard-workflows/utils/genprepMLWater.py", line 361, in genprepmlwater
    X = img_data.stack(z=['x','y']).to_array().transpose() # stack into transposed 2-d arr
  File "/opt/conda/lib/python3.7/site-packages/xarray/core/dataset.py", line 3329, in stack
    result = result._stack_once(dims, new_dim)
  File "/opt/conda/lib/python3.7/site-packages/xarray/core/dataset.py", line 3276, in _stack_once
    stacked_var = exp_var.stack(**{new_dim: dims})
  File "/opt/conda/lib/python3.7/site-packages/xarray/core/variable.py", line 1446, in stack
    result = result._stack_once(dims, new_dim)
  File "/opt/conda/lib/python3.7/site-packages/xarray/core/variable.py", line 1413, in _stack_once
    new_data = reordered.data.reshape(new_shape)
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/d

Prediction error


KeyboardInterrupt: 

# **Walkthroughs**

In [12]:
import yaml
import glob
import rasterio
from rasterio.crs import CRS
# from rasterio.transform import Affine
from affine import Affine
import rioxarray
import xarray as xr
from matplotlib import pyplot as plt
from datetime import datetime
from subprocess import Popen, PIPE, STDOUT
import pandas as pd
import os
import numpy as np
import shutil
import logging
import logging.handlers
from dateutil.parser import parse
import uuid
import geopandas as gpd
import rasterio
import rasterio.features
import gdal
import gc
import traceback

# ml stuff
from sklearn_xarray import wrap
from sklearn.ensemble import RandomForestClassifier

from utils.dc_water_classifier import wofs_classify
from utils.dc_clean_mask import landsat_qa_clean_mask
from utils.prep_utils import s3_list_objects, s3_download, s3_upload_cogs, create_yaml, cog_translate, get_geometry
from utils.dc_import_export import export_xarray_to_geotiff

In [13]:
def resamp_bands(xr, xrs):
    if xr.attrs['res'] == xrs[0].attrs['res']:
        return xr
    else:
        return xr.interp(x=xrs[0]['x'], y=xrs[0]['y'], method='nearest') # nearest as exclusively upsampling
    
def rename_bands(in_xr, des_bands, position):
    in_xr.name = des_bands[position]
    return in_xr

def get_valid(ds, prod):
    # Identify pixels with valid data
    if 'LANDSAT_8' in prod:
        good_quality = (
            (ds.pixel_qa == 322)  | # clear
            (ds.pixel_qa == 386)  |
            (ds.pixel_qa == 834)  |
            (ds.pixel_qa == 898)  |
            (ds.pixel_qa == 1346) |
            (ds.pixel_qa == 324)  | # water
            (ds.pixel_qa == 388)  |
            (ds.pixel_qa == 836)  |
            (ds.pixel_qa == 900)  |
            (ds.pixel_qa == 1348)
        )
    elif prod in ["LANDSAT_7", "LANDSAT_5", "LANDSAT_4"]:    
        good_quality = (
            (ds.pixel_qa == 66)   | # clear
            (ds.pixel_qa == 130)  |
            (ds.pixel_qa == 68)   | # water
            (ds.pixel_qa == 132)  
        )
    elif 'SENTINEL_2' in prod:
        good_quality = (
            (ds.scene_classification == 2) | # mask in DARK_AREA_PIXELS
#             (ds.scene_classification == 3) | # mask in CLOUD_SHADOWS
            (ds.scene_classification == 4) | # mask in VEGETATION
            (ds.scene_classification == 5) | # mask in NOT_VEGETATED
            (ds.scene_classification == 6) | # mask in WATER
            (ds.scene_classification == 7)   # mask in UNCLASSIFIED
        )
    elif 'WOFS_SUMMARY' in prod:
        good_quality = (
            (ds >= 0)
        )
    elif 'SENTINEL_1' in prod:
        good_quality = (
            (ds.vv != 0)
        )
    return good_quality

def get_ref_channel(prod):
    if ('LANDSAT' in prod) | ('SENTINEL_2' in prod): return 'swir1'
    elif 'SENTINEL_1' in prod: return 'vv'
    
def get_qa_channel(prod):
    if 'LANDSAT' in prod: return 'pixel_qa'
    elif 'SENTINEL_2' in prod: return 'scene_classification'
    elif 'SENTINEL_1' in prod: return 'layovershadow_mask'
    

def band_name_water(prod_path):
    """
    Determine l8 band of individual product from product name
    from path to specific product file
    """

    prod_name = os.path.basename(prod_path)
    parts = prod_name.split('_')
    prod_name = f"{parts[-2]}_{parts[-1][:-4]}"

    prod_map = {
        "watermask": 'watermask',
        "waterprob": 'waterprob'
    }
    layer_name = prod_map[prod_name]
    return layer_name

def yaml_prep_water(scene_dir, original_yml):
    """
    Prepare individual wofs directory containing L8/S2/S1 cog water products.
    """
    # scene_name = scene_dir.split('/')[-2][:26]
    scene_name = scene_dir.split('/')[-2]
    print ( "Preparing scene {}".format(scene_name) )
    print ( "Scene path {}".format(scene_dir) )
    
    # find all cog prods
    prod_paths = glob.glob(scene_dir + '*water*.tif')
    # print ( 'paths: {}'.format(prod_paths) )
    # for i in prod_paths: print ( i )
    
    # date time assumed eqv for start and stop - this isn't true and could be 
    # pulled from .xml file (or scene dir) not done yet for sake of progression
    t0=parse(str(datetime.strptime(original_yml['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')))
    # print ( t0 )
    t1=t0
    # print ( t1 )
    
    # name image product
    images = {
        prod_path.split('_')[-1][:9]: {
            'path': str(prod_path.split('/')[-1])
        } for prod_path in prod_paths
    }
    print ( images )

    # trusting bands coaligned, use one to generate spatial bounds for all
    projection, extent = get_geometry(os.path.join(str(scene_dir), images['watermask']['path']))
#     extent = 
    print(projection, extent)
    
    new_id = str(uuid.uuid5(uuid.NAMESPACE_URL, f"{scene_name}_water"))
    
    return {
        'id': new_id,
        'processing_level': original_yml['processing_level'],
        'product_type': "mlwater",
        'creation_dt': str(datetime.today().strftime('%Y-%m-%d %H:%M:%S')),
        'platform': {  
            'code': original_yml['platform']['code']
        },
        'instrument': {
            'name': original_yml['instrument']['name']
        },
        'extent': {
            'coord': original_yml['extent']['coord'],
            'from_dt': str(t0),
            'to_dt': str(t1),
            'center_dt': str(t0 + (t1 - t0) / 2)
        },
        'format': {
            'name': 'GeoTiff'
        },
        'grid_spatial': {
            'projection': projection
        },
        'image': {
            'bands': images
        },
        'lineage': {
            'source_datasets': original_yml['lineage']['source_datasets'],
        }  
    }


def mlwater(optical_yaml_path, summary_yaml_path, inter_dir='../data/', s3_bucket='public-eo-data', s3_dir='common_sensing/fiji/mlwater_test/',
            mask=None, output_crs=None):
    """
    optical_yaml_path: dc yml metadata of single image within S3 bucket
    summary_yaml_path: dc yml metadata of wofs-like summary product within S3 bucket
    """

    try: 
        
        scene_name = os.path.dirname(optical_yaml_path).split('/')[-1]
        
        inter_dir = f"{inter_dir}{scene_name}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        cog_dir = f"{inter_dir}{scene_name}/"
        os.makedirs(cog_dir, exist_ok=True)
        
        yml = f'{inter_dir}datacube-metadata.yaml'
        yml_summary = f'{inter_dir}datacube-metadata_watersummary.yaml'
        
        des_band_refs = {
            "LANDSAT_8": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
            "LANDSAT_7": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
            "LANDSAT_5": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
            "LANDSAT_4": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
            "SENTINEL_2": ['blue','green','red','nir','swir1','swir2','scene_classification'],
            "SENTINEL_1": ['vv','vh','layovershadow_mask'],
            "WOFS_SUMMARY": ['pc']}
        
        if not os.path.exists(yml):
            s3_download(s3_bucket, optical_yaml_path, yml)
            with open (yml) as stream: yml_meta = yaml.safe_load(stream)
            satellite = yml_meta['platform']['code'] # helper to generalise masking 
            des_bands = des_band_refs[satellite]
            print(satellite, des_bands)
            band_paths_s3 = [os.path.dirname(optical_yaml_path)+'/'+yml_meta['image']['bands'][b]['path'] for b in des_bands ]
            band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
            for s3, loc in zip(band_paths_s3, band_paths_local): 
                if not os.path.exists(loc):
                    s3_download(s3_bucket, s3, loc)
        elif os.path.exists(yml):
            with open (yml) as stream: yml_meta = yaml.safe_load(stream)
            satellite = yml_meta['platform']['code'] # helper to generalise masking 
            des_bands = des_band_refs[satellite]
            print(satellite, des_bands)
        else:
            print('boo')

        # FIND & DOWNLOAD WATER SUMMARY DATA
        if not os.path.exists(yml_summary):
            s3_download(s3_bucket, summary_yaml_path, yml_summary)
            with open (yml_summary) as stream: yml_summary_meta = yaml.safe_load(stream)
            summary = yml_summary_meta['platform']['code']
            des_bands_summary = des_band_refs[summary]
            print(summary, des_bands_summary)
            band_paths_s3 = [os.path.dirname(summary_yaml_path)+'/'+yml_summary_meta['image']['bands'][b]['path'] for b in des_bands_summary ]
            band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
            for s3, loc in zip(band_paths_s3, band_paths_local): 
                if not os.path.exists(loc):
                    s3_download(s3_bucket, s3, loc)
        elif os.path.exists(yml_summary):
            s3_download(s3_bucket, summary_yaml_path, yml_summary)
            with open (yml_summary) as stream: yml_summary_meta = yaml.safe_load(stream)
            summary = yml_summary_meta['platform']['code']
            des_bands_summary = des_band_refs[summary]
            print(summary, des_bands_summary)
            band_paths_s3 = [os.path.dirname(summary_yaml_path)+'/'+yml_summary_meta['image']['bands'][b]['path'] for b in des_bands_summary ]
            band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
            for s3, loc in zip(band_paths_s3, band_paths_local): 
                if not os.path.exists(loc):
                    s3_download(s3_bucket, s3, loc)
        else:
            print('boo')

        ref_channel = get_ref_channel(satellite)
        qa_channel = get_qa_channel(satellite)
        print(ref_channel, qa_channel)

        # LOAD & PREP IMAGE DATA
        o_bands_data = [ xr.open_rasterio(inter_dir + yml_meta['image']['bands'][b]['path']) for b in des_bands ] # loading
        o_bands_data = [ resamp_bands(i, o_bands_data) for i in o_bands_data ] # resamp 2 match band 1
        bands_data = xr.merge([rename_bands(bd, des_bands, i) for i,bd in enumerate(o_bands_data)]).rename({'band': 'time'}) # ensure band names & dims consistent
        bands_data = bands_data.assign_attrs(o_bands_data[0].attrs) # crs etc. needed later
        bands_data['time'] = [datetime.strptime(yml_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')] # time dim needed for wofs

        img_data = bands_data.isel(time = 0).drop(['time'])
        bands_data = None
        o_bands_data = None
        
        # LOAD & PREP WATER SUMMARY DATA
        o_bands_data = [ xr.open_rasterio(inter_dir + yml_summary_meta['image']['bands'][b]['path']) for b in des_bands_summary ] # loading
        o_bands_data = [ resamp_bands(i, o_bands_data) for i in o_bands_data ]
        sum_data = xr.merge([rename_bands(bd, des_bands_summary, i) for i,bd in enumerate(o_bands_data)]).rename({'band': 'time'}) # ensure band names & dims consistent
        sum_data = sum_data.assign_attrs(o_bands_data[0].attrs) # crs etc. needed later
        sum_data['time'] = [datetime.strptime(yml_summary_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')] # time dim needed for wofs

        class_data = sum_data.isel(time = 0).drop(['time']).pc
        sum_data = None
        o_bands_data = None
        
        # REPROJECT & ALIGN CRS+DIMS
        class_data = class_data.rio.reproject_match(img_data[ref_channel]) # repro + align grid
        img_data, class_data = xr.align(img_data, class_data, join="override") # force alignment of x,y precision
        
        if satellite == 'SENTINEL_1': # catch s1 and scall and re-fromat dtype
            att = img_data.attrs
            img_data = img_data*100
            img_data = img_data.astype('int16')
            img_data.attrs = att
        
        # VALID REGION MASKS
        clearskymask_img = get_valid(img_data, satellite) # img nd mask
        clearskymask_class = get_valid(class_data, summary) # water nd mask
        clearskymask_train = clearskymask_class.where(clearskymask_class == False, False) # empty mask
        clearskymask_train = clearskymask_train.where((clearskymask_img == False) | (clearskymask_class == False), True) # inner true mask
        
        # ASSIGN WATER/NON WATER CLASS LABELS
        water_thresh = 50 # 50% persistence in annual summary
        class_data = class_data.where((class_data < water_thresh) | (clearskymask_class == False), 100) # fix > prob to water
        class_data = class_data.where((class_data >= water_thresh) | (clearskymask_class == False), 0) # fix < prob to no water 
        
        # MASK TO TRAINING SAMPLES W/ IMPUTED ND
        train_data = img_data # dup as use img 4 implementation later
        train_data['waterclass'] = class_data # add channel for water mask
        train_data = train_data.where(clearskymask_train == True, -9999).drop([qa_channel]) # apply inner mask
        
        unique, counts = np.unique(train_data.waterclass, return_counts=True)
        if (counts[0] < 500) | (counts[1] < 5000):
            raise Exception(f'no class labels should be >5000 for ok classifier. no. training class samples: {counts[0]}{counts[1]}')
        
        # SPEC & TRAIN MODEL
        Y = train_data.waterclass.stack(z=['x','y']) # stack into 1-d arr
        X = train_data.drop(['waterclass']).stack(z=['x','y']).to_array().transpose() # stack into transposed 2-d arr
        # very shallow classifier - this is a super easy problem & we want it to be fast
        wrapper = wrap(RandomForestClassifier(n_estimators=2, 
                                       bootstrap = True,
                                       max_features = 'sqrt',
                                       max_depth=5,
                                       n_jobs=2,
                                       verbose=2
                                      ))
        wrapper.estimator.fit(X, Y) # do training
        
        # MASK TO FULL VALID IMAGE FOR IMPLEMENTATION
        img_data = img_data.drop([qa_channel,'waterclass']) # not sure how these ended up in here(?)
        img_data = img_data.where(clearskymask_img == True, -9999) # apply just the img mask this time
        
        # PREDICT + ASSIGN CONFIDENCE
        X = img_data.stack(z=['x','y']).to_array().transpose() # stack into transposed 2-d arr

        pred = wrapper.estimator.predict(X) # gen class predictions
        pred[pred==100] = 1 # refactor water from 100 to 1
        prob = wrapper.estimator.predict_proba(X)[:,2]*100 # gen confidence in assigned labels as int
        
        # RESHAPE OUTPUTS INTO IMAGE
        vars_0 = [i for i in X.transpose().to_dataset(dim='variable').data_vars] # get list of vars within img
        X_t = X.transpose().to_dataset(dim='variable') # recreate xrds (but no unstacking yet as need to drop in model outputs)
        X_t[vars_0[0]].data = pred # add class predictions as first channel
        X_t[vars_0[1]].data = prob # add confidence as second channel
        X_t = X_t.rename({vars_0[0]:'water_mask',vars_0[1]:'water_prob'}).drop(vars_0[2:]).unstack('z').transpose().astype('int16') # rename + drop vars + unstack xy dims back to 3-d xrds + transpose predictions back into correct orientation
        X_t = X_t.where(clearskymask_img,-9999) # ensure probs rm 4 nd regions
        X_t.attrs = img_data.attrs

        # EXPORT
        inter_prodir = inter_dir + scene_name + '_mlwater/'
        os.makedirs(inter_prodir, exist_ok=True)
        out_mask_prod = inter_prodir + scene_name + '_watermask.tif'
        out_prob_prod = inter_prodir + scene_name + '_waterprob.tif'
        output_crs = f"EPSG:{X_t.attrs['crs'].split(':')[-1]}"
        export_xarray_to_geotiff(X_t, out_mask_prod, bands=['water_mask'], crs=output_crs, x_coord='x', y_coord='y', no_data=-9999)
        export_xarray_to_geotiff(X_t, out_prob_prod, bands=['water_prob'], crs=output_crs, x_coord='x', y_coord='y', no_data=-9999)
        
        # CREATE YML
        create_yaml(inter_prodir, yaml_prep_water(inter_prodir, yml_meta)) # assumes majority of meta copied from original product yml
        
        # UPLOAD
        s3_upload_cogs(glob.glob(f'{inter_prodir}*'), s3_bucket, s3_dir)
        
        class_data = None
        att = None
        img_data = None
        clearskymask_img = None
        clearskymask_class = None
        clearskymask_train = None
        class_data = None
        train_data = None
        wrapperper = None
        X = None     
        Y = None
        pred = None
        prob = None
        vars_0 = None
        X_t = None
        
        gc.collect()
#         shutil.rmtree(inter_prodir)

    except Exception as e:
        print(e)
        traceback.print_exc()

        class_data = None
        att = None
        img_data = None
        clearskymask_img = None
        clearskymask_class = None
        clearskymask_train = None
        class_data = None
        train_data = None
        wrapperper = None
        X = None     
        Y = None
        pred = None
        prob = None
        vars_0 = None
        X_t = None

        gc.collect()
#         shutil.rmtree(inter_prodir)
        

In [14]:
p = l8_ymls[5:] + s1_ymls[:5] +s2_ymls[:5]

In [20]:
! mkdir /home/dataprep/data/masks/

In [21]:
! cp /home/dataprep/data/*/*/*mask* /home/dataprep/data/masks/

In [19]:
! ls -lh /home/dataprep/data/*/*/*mask*

-rwxr-xr-x 1 root root 115M Apr 20 09:28 /home/dataprep/data/LC08_L1TP_074072_20190113_tmp/LC08_L1TP_074072_20190113_mlwater/LC08_L1TP_074072_20190113_watermask.tif
-rwxr-xr-x 1 root root  306 Apr 20 09:42 /home/dataprep/data/LC08_L1TP_074072_20190113_tmp/LC08_L1TP_074072_20190113_mlwater/LC08_L1TP_074072_20190113_watermask.tif.aux.xml
-rwxr-xr-x 1 root root 115M Apr 20 09:31 /home/dataprep/data/LC08_L1TP_074072_20190129_tmp/LC08_L1TP_074072_20190129_mlwater/LC08_L1TP_074072_20190129_watermask.tif
-rwxr-xr-x 1 root root 115M Apr 20 09:33 /home/dataprep/data/LC08_L1TP_074072_20190214_tmp/LC08_L1TP_074072_20190214_mlwater/LC08_L1TP_074072_20190214_watermask.tif
-rwxr-xr-x 1 root root 115M Apr 20 09:36 /home/dataprep/data/LC08_L1TP_074072_20190302_tmp/LC08_L1TP_074072_20190302_mlwater/LC08_L1TP_074072_20190302_watermask.tif
-rwxr-xr-x 1 root root 115M Apr 20 09:39 /home/dataprep/data/LC08_L1TP_074072_20190318_tmp/LC08_L1TP_074072_20190318_mlwater/LC08_L1TP_074072_20190318_watermask.tif
-r

In [16]:
for i in l8_ymls[5:]:
    mlwater(i, wofssummary_ymls[0])

LANDSAT_8 ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']
WOFS_SUMMARY ['pc']
swir1 pixel_qa


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 2building tree 2 of 2



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   35.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.4s finished


Preparing scene LC08_L1TP_074072_20190403_mlwater
Scene path ../data/LC08_L1TP_074072_20190403_tmp/LC08_L1TP_074072_20190403_mlwater/
{'watermask': {'path': 'LC08_L1TP_074072_20190403_watermask.tif'}, 'waterprob': {'path': 'LC08_L1TP_074072_20190403_waterprob.tif'}}
{'spatial_reference': 'PROJCS["WGS 84 / UTM zone 60N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",177],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32660"]]', 'geo_ref_points': {'ul': {'x': 584400.0, 'y': -1802100.0}, 'ur': {'x': 815100.0, 'y': -1802100.0}, 'll': {'x'

...

Preparing scene LC08_L1TP_074072_20190825_mlwater
Scene path ../data/LC08_L1TP_074072_20190825_tmp/LC08_L1TP_074072_20190825_mlwater/
{'watermask': {'path': 'LC08_L1TP_074072_20190825_watermask.tif'}, 'waterprob': {'path': 'LC08_L1TP_074072_20190825_waterprob.tif'}}
{'spatial_reference': 'PROJCS["WGS 84 / UTM zone 60N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",177],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32660"]]', 'geo_ref_points': {'ul': {'x': 584100.0, 'y': -1802100.0}, 'ur': {'x': 814800.0, 'y': -1802100.0}, 'll': {'x'

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 2
building tree 2 of 2


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   16.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.2s finished


Preparing scene LC08_L1TP_074072_20190910_mlwater
Scene path ../data/LC08_L1TP_074072_20190910_tmp/LC08_L1TP_074072_20190910_mlwater/
{'watermask': {'path': 'LC08_L1TP_074072_20190910_watermask.tif'}, 'waterprob': {'path': 'LC08_L1TP_074072_20190910_waterprob.tif'}}
{'spatial_reference': 'PROJCS["WGS 84 / UTM zone 60N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",177],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32660"]]', 'geo_ref_points': {'ul': {'x': 584100.0, 'y': -1802100.0}, 'ur': {'x': 815100.0, 'y': -1802100.0}, 'll': {'x'

# Walkthrough

In [23]:
optical_yaml_path = l8_ymls[1]
# optical_yaml_path = s2_ymls[8]
# optical_yaml_path = s1_ymls[8]
# summary_yaml_path = wofssummary_ymls[0]

In [24]:
# essential input vars
s3_bucket = 'public-eo-data'
s3_dir = 'common_sensing/fiji/mlwater_test/'
inter_dir = '../data/'
coast_buffer = 'common_sensing/ancillary_products/Buffers/Fiji_1kmBuffer_shp'

scene_name = os.path.dirname(optical_yaml_path).split('/')[-1]

inter_dir = f"{inter_dir}{scene_name}_tmp/"
os.makedirs(inter_dir, exist_ok=True)
cog_dir = f"{inter_dir}{scene_name}/"
os.makedirs(cog_dir, exist_ok=True)

yml = f'{inter_dir}datacube-metadata.yaml'
yml_summary = f'{inter_dir}datacube-metadata_watersummary.yaml'

des_band_refs = {
    "LANDSAT_8": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
    "LANDSAT_7": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
    "LANDSAT_5": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
    "LANDSAT_4": ['blue','green','red','nir','swir1','swir2','pixel_qa'],
    "SENTINEL_2": ['blue','green','red','nir','swir1','swir2','scene_classification'],
    "SENTINEL_1": ['vv','vh','layovershadow_mask'],
    "WOFS_SUMMARY": ['pc']}


In [25]:
# FIND & DOWNLOAD IMAGE DATA
if not os.path.exists(yml):
    s3_download(s3_bucket, optical_yaml_path, yml)
    with open (yml) as stream: yml_meta = yaml.safe_load(stream)
    satellite = yml_meta['platform']['code'] # helper to generalise masking 
    des_bands = des_band_refs[satellite]
    print(satellite, des_bands)
    band_paths_s3 = [os.path.dirname(optical_yaml_path)+'/'+yml_meta['image']['bands'][b]['path'] for b in des_bands ]
    band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
    for s3, loc in zip(band_paths_s3, band_paths_local): 
        if not os.path.exists(loc):
            s3_download(s3_bucket, s3, loc)
elif os.path.exists(yml):
    with open (yml) as stream: yml_meta = yaml.safe_load(stream)
    satellite = yml_meta['platform']['code'] # helper to generalise masking 
    des_bands = des_band_refs[satellite]
    print(satellite, des_bands)
else:
    print('boo')

# FIND & DOWNLOAD WATER SUMMARY DATA
if not os.path.exists(yml_summary):
    s3_download(s3_bucket, summary_yaml_path, yml_summary)
    with open (yml_summary) as stream: yml_summary_meta = yaml.safe_load(stream)
    summary = yml_summary_meta['platform']['code']
    des_bands_summary = des_band_refs[summary]
    print(summary, des_bands_summary)
    band_paths_s3 = [os.path.dirname(summary_yaml_path)+'/'+yml_summary_meta['image']['bands'][b]['path'] for b in des_bands_summary ]
    band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
    for s3, loc in zip(band_paths_s3, band_paths_local): 
        if not os.path.exists(loc):
            s3_download(s3_bucket, s3, loc)
elif os.path.exists(yml_summary):
    s3_download(s3_bucket, summary_yaml_path, yml_summary)
    with open (yml_summary) as stream: yml_summary_meta = yaml.safe_load(stream)
    summary = yml_summary_meta['platform']['code']
    des_bands_summary = des_band_refs[summary]
    print(summary, des_bands_summary)
    band_paths_s3 = [os.path.dirname(summary_yaml_path)+'/'+yml_summary_meta['image']['bands'][b]['path'] for b in des_bands_summary ]
    band_paths_local = [inter_dir+os.path.basename(i) for i in band_paths_s3]
    for s3, loc in zip(band_paths_s3, band_paths_local): 
        if not os.path.exists(loc):
            s3_download(s3_bucket, s3, loc)
else:
    print('boo')

ref_channel = get_ref_channel(satellite)
qa_channel = get_qa_channel(satellite)
print(ref_channel, qa_channel)

LANDSAT_8 ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']
WOFS_SUMMARY ['pc']
swir1 pixel_qa


In [26]:
summary, des_bands_summary, ref_channel

('WOFS_SUMMARY', ['pc'], 'swir1')

In [27]:
# LOAD & PREP IMAGE DATA
o_bands_data = [ xr.open_rasterio(inter_dir + yml_meta['image']['bands'][b]['path']) for b in des_bands ] # loading
o_bands_data = [ resamp_bands(i, o_bands_data) for i in o_bands_data ] # resamp 2 match band 1
bands_data = xr.merge([rename_bands(bd, des_bands, i) for i,bd in enumerate(o_bands_data)]).rename({'band': 'time'}) # ensure band names & dims consistent
bands_data = bands_data.assign_attrs(o_bands_data[0].attrs) # crs etc. needed later
bands_data['time'] = [datetime.strptime(yml_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')] # time dim needed for wofs

img_data = bands_data.isel(time = 0).drop(['time'])
bands_data = None
o_bands_data = None

In [28]:
# LOAD & PREP WATER SUMMARY DATA
o_bands_data = [ xr.open_rasterio(inter_dir + yml_summary_meta['image']['bands'][b]['path']) for b in des_bands_summary ] # loading
o_bands_data = [ resamp_bands(i, o_bands_data) for i in o_bands_data ]
sum_data = xr.merge([rename_bands(bd, des_bands_summary, i) for i,bd in enumerate(o_bands_data)]).rename({'band': 'time'}) # ensure band names & dims consistent
sum_data = sum_data.assign_attrs(o_bands_data[0].attrs) # crs etc. needed later
sum_data['time'] = [datetime.strptime(yml_summary_meta['extent']['center_dt'], '%Y-%m-%d %H:%M:%S')] # time dim needed for wofs

class_data = sum_data.isel(time = 0).drop(['time']).pc
sum_data = None
o_bands_data = None

In [29]:
# REPROJECT & ALIGN CRS+DIMS
class_data = class_data.rio.reproject_match(img_data[ref_channel]) # repro + align grid
img_data, class_data = xr.align(img_data, class_data, join="override") # force alignment of x,y precision

In [30]:
if satellite == 'SENTINEL_1': # catch s1 and scall and re-fromat dtype
    att = img_data.attrs
    img_data = img_data*100
    img_data = img_data.astype('int16')
    img_data.attrs = att

In [31]:
# VALID REGION MASKS
clearskymask_img = get_valid(img_data, satellite) # img nd mask
clearskymask_class = get_valid(class_data, summary) # water nd mask
clearskymask_train = clearskymask_class.where(clearskymask_class == False, False) # empty mask
clearskymask_train = clearskymask_train.where((clearskymask_img == False) | (clearskymask_class == False), True) # inner true mask

In [32]:
# ASSIGN WATER/NON WATER CLASS LABELS
water_thresh = 50 # 50% persistence in annual summary
class_data = class_data.where((class_data < water_thresh) | (clearskymask_class == False), 100) # fix > prob to water
class_data = class_data.where((class_data >= water_thresh) | (clearskymask_class == False), 0) # fix < prob to no water 

In [33]:
# MASK TO TRAINING SAMPLES W/ IMPUTED ND
train_data = img_data # dup as use img 4 implementation later
train_data['waterclass'] = class_data # add channel for water mask
train_data = train_data.where(clearskymask_train == True, -9999).drop([qa_channel]) # apply inner mask

In [34]:
unique, counts = np.unique(train_data.waterclass, return_counts=True)

if (counts[0] < 500) | (counts[1] < 5000):
    raise Exception(f'no class labels should be >5000 for ok classifier. no. training class samples: {counts[0]}{counts[1]}')

In [35]:
# SPEC & TRAIN MODEL
Y = train_data.waterclass.stack(z=['x','y']) # stack into 1-d arr
X = train_data.drop(['waterclass']).stack(z=['x','y']).to_array().transpose() # stack into transposed 2-d arr

# very shallow classifier - this is a super easy problem & we want it to be fast
wrapper = wrap(RandomForestClassifier(n_estimators=4, 
                               bootstrap = True,
                               max_features = 'sqrt',
                               max_depth=5,
                               n_jobs=2,
                               verbose=2
                              ))
wrapper.estimator.fit(X, Y) # do training

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 2building tree 2 of 2



[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   32.3s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2, n_jobs=2,
                       oob_score=False, random_state=None, verbose=2,
                       warm_start=False)

In [36]:
# MASK TO FULL VALID IMAGE FOR IMPLEMENTATION
img_data = img_data.drop([qa_channel,'waterclass']) # not sure how these ended up in here(?)
img_data = img_data.where(clearskymask_img == True, -9999) # apply just the img mask this time

In [37]:
# PREDICT + ASSIGN CONFIDENCE
X = img_data.stack(z=['x','y']).to_array().transpose() # stack into transposed 2-d arr

pred = wrapper.estimator.predict(X) # gen class predictions
pred[pred==100] = 1
prob = wrapper.estimator.predict_proba(X)[:,1]*100 # gen confidence in assigned labels as int

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.4s finished


In [38]:
# RESHAPE OUTPUTS INTO IMAGE
vars_0 = [i for i in X.transpose().to_dataset(dim='variable').data_vars] # get list of vars within img
X_t = X.transpose().to_dataset(dim='variable') # recreate xrds (but no unstacking yet as need to drop in model outputs)
X_t[vars_0[0]].data = pred # add class predictions as first channel
X_t[vars_0[2]].data = prob # add confidence as second channel
X_t = X_t.rename({vars_0[0]:'water_mask',vars_0[1]:'water_prob'}).drop(vars_0[2:]).unstack('z').transpose().astype('int16') # rename + drop vars + unstack xy dims back to 3-d xrds + transpose predictions back into correct orientation
X_t = X_t.where(clearskymask_img,-9999) # ensure probs rm 4 nd regions
X_t.attrs = img_data.attrs

In [39]:
inter_prodir = inter_dir + scene_name + '_mlwater/'
os.makedirs(inter_prodir, exist_ok=True)
out_mask_prod = inter_prodir + scene_name + '_watermask.tif'
out_prob_prod = inter_prodir + scene_name + '_waterprob.tif'
output_crs = f"EPSG:{X_t.attrs['crs'].split(':')[-1]}"
export_xarray_to_geotiff(X_t, out_mask_prod, bands=['water_mask'], crs=output_crs, x_coord='x', y_coord='y', no_data=-9999)
export_xarray_to_geotiff(X_t, out_prob_prod, bands=['water_prob'], crs=output_crs, x_coord='x', y_coord='y', no_data=-9999)

In [ ]:
create_yaml(inter_prodir, yaml_prep_water(inter_prodir, yml_meta)) # assumes majority of meta copied from original product yml

In [ ]:
s3_upload_cogs(glob.glob(f'{inter_prodir}*'), s3_bucket, s3_dir)

In [ ]:
# Index
cmd = f'python3 ./utils/dataset_index_from_s3_bucket.py {s3_bucket} -p {s3_dir}{scene_name + "_mlwater"}/ --endpoint_url="http://s3-uk-1.sa-catapult.co.uk" --unsigned_requests --start_date 1960-01-01 --end_date 2030-01-01'
p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
out = p.stdout.read()
print(out)

In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
ax.imshow(img_data.where(clearskymask_img)[ref_channel])

In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
ax.imshow(X_t.where(X_t.water_mask>=0).water_mask)

In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
ax.imshow(X_t.where(X_t.water_prob>=0).water_prob)